<a href="https://colab.research.google.com/github/Biline-dev/IARN/blob/main/TP1_IARN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
xyz = pd.read_csv('label.csv') 
print(xyz.head(1))
xyz = pd.read_csv('data.csv') 
print(xyz.head(1))

   10
0  10
   13  13.1  13.2  13.3  13.4  ...  13.216  13.217  13.218  13.219  13.220
0  14    14    14    14    14  ...      14      14      14      14      14

[1 rows x 400 columns]
